In [2]:
# R code 4.24

library(rethinking)
data(Howell1)

d <- Howell1
d2 <- d[ d$age >= 18, ]

Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.12.1, packaged: 2016-09-11 13:07:50 UTC, GitRev: 85f7a56811da)
For execution on a local, multicore CPU with excess RAM we recommend calling
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())
Loading required package: parallel
rethinking (Version 1.59)


In [3]:
flist <- alist (
    height ~ dnorm(mu, sigma),
    mu ~ dnorm(178, 20),
    sigma ~ dunif(0, 50)
)

m4.1 <- map(flist, data=d2)

In [4]:
precis(m4.1)

        Mean StdDev   5.5%  94.5%
mu    154.61   0.41 153.95 155.27
sigma   7.73   0.29   7.26   8.20

These number provide Gaussian approximations (normal/quadratic) for each parameter's marginal distribution. This means the plausibility of each value of $\mu$, after averaging over the plausibilities of each value of $\sigma$ is given by a Gaussian distribution with mean 154.6 and standard deviation 0.4

In [5]:
m4.2 <- map(
    alist(
        height ~ dnorm(mu, sigma),
        mu ~ dnorm(178, 0.1),
        sigma ~ dunif(0, 50)
    ),
    data = d2
)

In [6]:
precis(m4.2)

        Mean StdDev   5.5%  94.5%
mu    177.86   0.10 177.70 178.02
sigma  24.52   0.93  23.03  26.00

Notice that the estimate for $\mu$ has hardly moved off the prior. The prior was very concentrated around 178, so this is not surprising. But also notice the estimate for $\sigma$ has changed quite a lot, even though we didn't change its prior at all. Once the golem is certain that the mean is 178 -- as the prior insists -- then the golem has to estimate $\sigma$ conditional on that fact. This results in a different posterior for $\sigma$, even though all we changed was the prior for $\mu$.

##### 4.3.6 Sampling from a ```map``` fit

Just like a mean and a variance are sufficient to describe a one-dimensional Gaussian distribution, a list of means and a matrix of variances and covariances are sufficient to describe a multi-dimensional Gaussian distribution. To see the matrix of variances and covariances for model m4.1: 

In [7]:
vcov(m4.1)

,mu,sigma
mu,0.1696800422,0.0002247144
sigma,0.0002247144,0.0848312814


A variance-covariance matrix can be factored into two elements:
- a vector of variances for the parameters
- a correlation matrix that tells us how changes in any parameter lead to correlated changes in the others

In [8]:
diag(vcov(m4.1))
cov2cor(vcov(m4.1))

mu      sigma 
0.16968004 0.08483128

,mu,sigma
mu,1.000000000,0.001872999
sigma,0.001872999,1.000000000


$$ var = std^2$$

The two element vector (```diag(vcov(m4.1))```) in the output is the list of variances. If you take the square root, you get the standard deviation of the parameters, as shown in the precis output.

In [9]:
sqrt(diag(vcov(m4.1)))

mu     sigma 
0.4119224 0.2912581

In [10]:
# R code 4.32
post <- extract.samples(m4.1, n=1e4)
head(post)

mu,sigma
154.4174,7.662421
154.7229,7.843495
154.6446,7.472021
154.9366,7.919850
153.8720,7.710660
155.2253,7.981645


In [11]:
# R code 4.33
precis(post)

        Mean StdDev  |0.89  0.89|
mu    154.60   0.41 153.94 155.26
sigma   7.73   0.29   7.26   8.19

#### Chapter 10: debugging PYMC3

In [14]:
data(UCBadmit)
d <- UCBadmit

In [15]:
show(d)

   dept applicant.gender admit reject applications
1     A             male   512    313          825
2     A           female    89     19          108
3     B             male   353    207          560
4     B           female    17      8           25
5     C             male   120    205          325
6     C           female   202    391          593
7     D             male   138    279          417
8     D           female   131    244          375
9     E             male    53    138          191
10    E           female    94    299          393
11    F             male    22    351          373
12    F           female    24    317          341


In [17]:
d$male <- ifelse( d$applicant.gender == "male" , 1, 0 )

In [18]:
m10.6 <- map (
    alist(
        admit ~ dbinom( applications, p),
        logit(p) <- a + bm * male,
        a ~ dnorm(0, 10),
        bm ~ dnorm(0, 10)
    ), 
    data = d 
)

m10.7 <- map (
    alist(
        admit ~ dbinom( applications, p),
        logit(p) <- a,
        a ~ dnorm(0, 10)
    ), 
    data = d 
)

In [19]:
compare(m10.6, m10.7)

        WAIC pWAIC dWAIC weight    SE   dSE
m10.6 5954.9   2.0   0.0      1 35.05    NA
m10.7 6046.2   0.9  91.4      0 29.89 19.19

In [20]:
precis(m10.6)

    Mean StdDev  5.5% 94.5%
a  -0.83   0.05 -0.91 -0.75
bm  0.61   0.06  0.51  0.71